In [1]:
import pandas as pd

In [2]:
firms = pd.read_excel('firm_names.xlsx')
scores = pd.read_excel('Soc.xlsx', index_col = 0)
returns = pd.read_excel('monthlyreturns.xlsx', index_col = 0)
eu_countrycodes = ['AL','AD', 'AM','AT','BA','BE','BG','CH','CY','DE','DK','EE','ES','FI','FR','GE', 'GB','GR','HR','HU','IE','IS','IT','LT','LV','MC','MK','MT','NL','NO','PL','PT','RO','RS', 'RU','SE','SI', 'TR', 'UA', 'MD', 'LI']

Transposing table to get firms as rows, reset columns to dates

In [5]:
scores = scores.transpose()
scores.index.rename('ISIN', inplace=True)

firms.set_index("ISIN", inplace=True)

Join Social scores to firm's names and locations
Filter on firms that are located in Europe

In [6]:
scores = scores.join(firms, how="left", on="ISIN")
scores = scores[scores["Country"].isin(eu_countrycodes)]

In [7]:
scores.dropna(how='all', axis='columns', inplace=True)

In [8]:
eu_firm_returns = returns[scores.index]

eu_firm_returns

,AT000000STR1,AT00000VIE62,AT0000606306,AT0000642806,AT0000652011,AT0000720008,AT0000730007,AT0000743059,AT0000746409,AT0000767553,...,RU000A0DKVS5,US68370R1095,US69338N2062,US7171402065,RU000A0JRKT8,RU000A0B6NK6,US92927K1025,US97263M1099,US98387E2054,VGG1890L1076
2002-01-31,NaN,0.004704,NaN,NaN,0.044299,0.028358,-0.031736,0.017735,0.011466,-0.000433,...,NaN,0.009666,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002-02-28,NaN,0.056424,NaN,NaN,-0.053178,0.016936,-0.100985,0.001924,0.036171,0.105286,...,NaN,0.143159,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002-03-31,NaN,0.013897,NaN,NaN,0.102579,-0.055194,0.000246,0.012976,-0.027588,0.141121,...,NaN,0.042653,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2002-04-30,NaN,0.099059,NaN,NaN,0.130746,0.032542,0.279030,0.131208,0.052367,-0.127104,...,NaN,0.012889,NaN,NaN,NaN,NaN,NaN,0.069556,NaN,NaN
2002-05-31,NaN,0.085816,NaN,NaN,0.198617,0.019151,0.177818,-0.015824,0.103447,0.035419,...,NaN,-0.289719,NaN,NaN,NaN,NaN,NaN,-0.033756,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-08-31,0.121001,0.063535,-0.050259,0.042363,-0.035623,0.079594,-0.037375,-0.039665,0.144400,0.042345,...,0.003129,0.000000,0.0,0.0,-0.006342,-0.040000,0.0,0.000000,0.070900,-0.086764
2020-09-30,0.062159,0.043616,0.045772,0.019299,0.066608,-0.007878,-0.048887,0.011063,0.022864,0.019233,...,0.038839,0.000000,0.0,0.0,0.034043,-0.020833,0.0,0.000000,-0.070410,0.129211
2020-10-31,-0.039832,-0.141218,-0.167263,-0.019134,-0.144955,-0.022696,-0.054567,-0.178596,0.027758,-0.019080,...,-0.100450,0.000000,0.0,0.0,-0.020576,-0.021277,0.0,0.000000,0.038995,0.144715
2020-11-30,-0.027859,-0.114198,-0.038687,-0.008375,0.001620,-0.026740,0.114400,-0.043243,0.043868,-0.008422,...,-0.052746,0.000000,0.0,0.0,-0.031513,-0.108696,0.0,0.000000,-0.034274,0.217025
